In [26]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns
import altair as alt

In [27]:
data_path = ''
filename = 'Global_Mobility_Report.zip'

mob = pd.read_csv(
    os.path.join(data_path, filename),
    compression='zip',
    parse_dates=['date'],
    low_memory=False
)

In [5]:
# truncate columns names
mob = mob.rename(columns={
    'retail_and_recreation_percent_change_from_baseline': 'retail_recreation',
    'grocery_and_pharmacy_percent_change_from_baseline': 'grocery_pharmacy',
    'parks_percent_change_from_baseline': 'parks',
    'transit_stations_percent_change_from_baseline': 'transit_stations',
    'workplaces_percent_change_from_baseline': 'workplaces',
    'residential_percent_change_from_baseline': 'residential'
})

In [6]:
# filter
mobs = mob.loc[
    mob.census_fips_code == 8013.0 # Boulder
].reset_index(drop=True)

In [7]:
def highlight_weekends(timeseries):
    """
    Shades in weekend days along axis.
    """
    # access datetime attributes of timeseries
    dt = timeseries.dt
    # create a series of weekend days
    weekends = timeseries[dt.dayofweek >= 5].groupby([dt.year, dt.isocalendar().week]).agg(['min', 'max'])
    # flatten multi index (altair doesn't like it)
    weekends.index = weekends.index.to_flat_index()
    return weekends

In [25]:
activity='workplaces'

line = alt.Chart(mobs).mark_area(
    line={'color':'seagreen'},
    color='seagreen',
    opacity=0.1
).encode(
    alt.X('date:T', title='date'),
    alt.Y(f'{activity}:Q', title='percent_change'),
    tooltip=[alt.Tooltip('date'), alt.Tooltip('workplaces', title='percent_change')]
).interactive()

# create a horizontal line at baseline
# line2 = alt.Chart(pd.DataFrame({'y': [0]})).mark_rule().encode(y='y')

weekends = highlight_weekends(mobs.date)

rect = alt.Chart(weekends).mark_rect(
    opacity=0.2,
    color='gray'
).encode(
    x='min:T',
    x2='max:T'
)

(rect + line).properties(
    width=1000,
    height=500,
    title=[f'Google mobility report - {activity}', '(shaded bars=weekends)'] # pass array of strings for multiline
).configure_axis(
    grid=False
).configure_view(
    strokeWidth=0
)

alt.LayerChart(...)